<a href="https://colab.research.google.com/github/JEN6YT/APS360-Project/blob/main/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageDraw
import pandas as pd
import os
import torchvision.transforms as transforms
import torch
import random
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Get image and label


In [ ]:
# read the xlsx file from Google Drive
file_path = '/content/drive/My Drive/U of T/APS360 Deep Learning/NIH-NLM-ThinBloodSmearsPf/img_path.xlsx'

# The dataframe store all the path, i.e. 'NIH-NLM-ThinBloodSmearsPf\Polygon Set\142C38P...'
df1 = pd.read_excel(file_path)

# create complete path for each image
prefix = "/content/drive/My Drive/U of T/APS360 Deep Learning/"
df1 = df1.applymap(lambda x: prefix + str(x))


In [ ]:
df1.head()

,File path
0,/content/drive/My Drive/U of T/APS360 Deep Lea...
1,/content/drive/My Drive/U of T/APS360 Deep Lea...
2,/content/drive/My Drive/U of T/APS360 Deep Lea...
3,/content/drive/My Drive/U of T/APS360 Deep Lea...
4,/content/drive/My Drive/U of T/APS360 Deep Lea...


In [ ]:
df1.shape

(800, 1)

In [ ]:
# read the xlsx file from Google Drive
file_path_infected = '/content/drive/My Drive/U of T/APS360 Deep Learning/NIH-NLM-ThinBloodSmearsPf/infected_RBC.xlsx'

# The dataframe store all the path, i.e. 'NIH-NLM-ThinBloodSmearsPf\Polygon Set\142C38P...'
df2 = pd.read_excel(file_path_infected)

In [ ]:
df2.head()

,Infected RBC
0,39
1,27
2,42
3,37
4,45


In [ ]:
df = pd.merge(df1, df2, left_index=True, right_index=True)

In [ ]:
df.head()

,File path,Infected RBC
0,/content/drive/My Drive/U of T/APS360 Deep Lea...,39
1,/content/drive/My Drive/U of T/APS360 Deep Lea...,27
2,/content/drive/My Drive/U of T/APS360 Deep Lea...,42
3,/content/drive/My Drive/U of T/APS360 Deep Lea...,37
4,/content/drive/My Drive/U of T/APS360 Deep Lea...,45


In [ ]:
df.shape

(800, 2)

## Label, resize, save in array

In [ ]:
data_list = []
target_size = (224, 224)

# iterate over rows and label images
for i, row in df.iterrows():
  # get the file path and infected RBC count
  print(i)

  file_path = row['File path']
  infected_rbc = row['Infected RBC']
  
  # open the image using PIL
  image = Image.open(file_path)
  image = image.resize(target_size)
  img_arr = np.array(image)
  
  # label the image based on the infected RBC count
  if infected_rbc == 0:
    label = 0
  elif infected_rbc > 0:
    label = 1
  
  # append the image and label to the list
  data = {'image': img_arr, 'label': label}
  data_list.append(data)

# convert the list to numpy array and save it
np.save('data.npy', np.array(data_list))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
data_list[0]

{'image': array([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 2],
         [1, 1, 3],
         [1, 1, 2]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [0, 0, 1],
         [1, 1, 1],
         [1, 1, 2]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [0, 0, 1],
         [1, 1, 1]],
 
        ...,
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [1, 1, 1],
         [1, 0, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [2, 0, 1]]], dtype=uint8), 'label': 1}

# Checking balance

In [ ]:
infected = 0
uninfected = 0
for data in data_list:
  if data['label'] == 1:
    infected += 1
  if data['label'] == 0:
    uninfected += 1
total = infected+uninfected
print(f'infected data {infected/total} and uninfected data {uninfected/total}')

infected data 0.71125 and uninfected data 0.28875


## Resampling

In [ ]:
# load the data from the saved numpy array
data = np.load('data.npy', allow_pickle=True)

# get the features (images) and labels
image = np.array([d['image'].flatten() for d in data])
label = np.array([d['label'] for d in data])

# reshape X to a 2D array
image = image.reshape(image.shape[0], -1)

# apply RandomOverSampler to X and y
ros = RandomOverSampler()
image_resampled, label_resampled = ros.fit_resample(image, label)

# combine X_resampled and y_resampled into a list of dicts
data_resampled = [{'image': image_resampled[i], 'label': label_resampled[i]} for i in range(len(label_resampled))]

# save the resampled data to a new numpy array
np.save('data_resampled.npy', np.array(data_resampled))

In [ ]:
infected = 0
uninfected = 0
for data in data_resampled:
  if data['label'] == 1:
    infected += 1
  if data['label'] == 0:
    uninfected += 1
total = infected+uninfected
print(f'infected data {infected/total} and uninfected data {uninfected/total}')

infected data 0.5 and uninfected data 0.5


In [ ]:
total

1138

# Normalization

In [ ]:
# Define the normalization function
def normalize(image):
  return (image - np.min(image)) / (np.max(image) - np.min(image))

# Normalize the data
normalized_data = []
for data in data_resampled:
  normalized_image = normalize(data['image'])
  normalized_data.append({'image': normalized_image, 'label': data['label']})

# Save the normalized data as a .npy file
np.save('normalized_data.npy', normalized_data)
print(normalized_data)

[{'image': array([0.00431034, 0.00431034, 0.00431034, ..., 0.00862069, 0.        ,
       0.00431034]), 'label': 1}, {'image': array([0.00416667, 0.00416667, 0.00416667, ..., 0.00833333, 0.        ,
       0.0125    ]), 'label': 1}, {'image': array([0.       , 0.       , 0.       , ..., 0.0041841, 0.0041841,
       0.0041841]), 'label': 1}, {'image': array([0.        , 0.        , 0.        , ..., 0.00425532, 0.00425532,
       0.00425532]), 'label': 1}, {'image': array([0.00425532, 0.00425532, 0.00425532, ..., 0.00425532, 0.00425532,
       0.00425532]), 'label': 1}, {'image': array([0.00413223, 0.00413223, 0.00413223, ..., 0.00826446, 0.        ,
       0.00826446]), 'label': 0}, {'image': array([0.00414938, 0.00414938, 0.01244813, ..., 0.00414938, 0.00414938,
       0.00414938]), 'label': 0}, {'image': array([0.00413223, 0.00413223, 0.00413223, ..., 0.00413223, 0.00413223,
       0.00413223]), 'label': 0}, {'image': array([0.00411523, 0.00411523, 0.00411523, ..., 0.00411523, 0.00411

In [ ]:
# get the features (images) and labels
X = np.array([d['image'] for d in normalized_data])
y = np.array([d['label'] for d in normalized_data])

print(X.shape)
print(len(y))

# Split your data into training, validation and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=66)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=66)

(1138, 150528)
1138


In [ ]:
X_train.shape

(557, 150528)

In [ ]:
# create an ImageDataGenerator instance with desired data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=25,
    horizontal_flip=True
)

# compute internal statistics of the data
datagen.fit(X_train)

# generate augmented data in batches
augmented_data_generator = datagen.flow(X_train, y_train, batch_size=32)

# iterate over batches of augmented data and add them to the original dataset
for x_batch, y_batch in augmented_data_generator:
  X_train = np.concatenate([X_train, x_batch], axis=0)
  y_train = np.concatenate([y_train, y_batch], axis=0)
  
  # break the loop if we have reached the desired number of samples
  if len(X_train) >= 1348:
      break

ValueError: ignored

# Baseline Model

Baseline Model: Random Forest

Create a RandomForestClassifier with 100 decision trees and train the model using the training data with rf.fit(X_train, y_train).

After training the model, we use it to predict the target variable for the test data with y_pred = rf.predict(X_test).

Finally, we evaluate the accuracy of the model using accuracy_score and print the result.

In [ ]:
# Import the necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest classifier with 10 decision trees
rf = RandomForestClassifier(n_estimators=10, random_state=42)

# flatten the input data
ytrain = y_train.reshape(-1,1)

# Train the Random Forest classifier
rf.fit(X_train, ytrain)

# Predict the target variable for the test data
y_pred = rf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


<ipython-input-72-7c290ea788b1>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, ytrain)


Accuracy: 0.8947368421052632
